In [ ]:
# Import necessary libraries
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose
from datasets import load_dataset
import numpy as np
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner, DirichletPartitioner
from flwr.client import NumPyClient
from flwr.common import Context, NDArrays, Scalar, ndarrays_to_parameters, EvaluateRes
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from collections import OrderedDict
from typing import Dict, Tuple, List
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import cohen_kappa_score, f1_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from flwr.server.client_proxy import ClientProxy
from flwr.common.logger import log
from logging import INFO

# Define constants
NUM_CLIENTS = 5
NUM_ROUNDS = 50
BATCH_SIZE = 32

# Define the number of attackers
ATTACKER_IDS = [0]  # For one attacker, set to [0]; for two attackers, set to [0, 1]

# Define whether to use IID or non-IID data
USE_IID = False  # Set to False for non-IID data

# Define partitioner based on IID or non-IID
if USE_IID:
    # IID Partitioning
    partitioner = IidPartitioner(num_partitions=NUM_CLIENTS)
else:
    # Non-IID Partitioning using Dirichlet distribution
    alpha = 0.1  # Smaller alpha means more heterogeneity
    partitioner = DirichletPartitioner(num_partitions=NUM_CLIENTS, alpha=alpha, partition_by="label")

# Load the CIFAR-10 dataset and partition it
fds = FederatedDataset(dataset="cifar10", partitioners={"train": partitioner})  # Changed from "ylecun/mnist" to "cifar10"

def get_cifar10_dataloaders(cifar10_dataset, batch_size: int):
    """
    Updated function to handle CIFAR-10 data loaders.
    Applies appropriate transformations for CIFAR-10 images.
    """
    # CIFAR-10 normalization parameters
    pytorch_transforms = Compose([
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    # Prepare transformation functions
    def apply_transforms(batch):
        """Apply transforms to the partition from FederatedDataset."""
        # Changed 'image' to 'img' to match CIFAR-10 dataset keys
        batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
        return batch

    cifar10_dataset = cifar10_dataset.with_transform(apply_transforms)

    # Construct PyTorch dataloader
    dataloader = DataLoader(cifar10_dataset, batch_size=batch_size, shuffle=True)
    return dataloader

# Define the neural network model suitable for CIFAR-10
class Net(nn.Module):
    def __init__(self, num_classes: int) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # Changed input channels from 1 to 3 for CIFAR-10
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # Adjusted input features from 16*4*4 to 16*5*5 for 32x32 images
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)  # Adjusted to match the new input size
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Functions to set and get model parameters
def set_params(model, parameters):
    """Replace model parameters with those passed as parameters."""
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.from_numpy(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

def get_params(model):
    """Extract model parameters as a list of NumPy arrays."""
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

# Training function with label flipping for attackers
def train(net, trainloader, optimizer, device="cpu", is_attacker=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    net.to(device)
    net.train()
    for batch in trainloader:
        images, labels = batch["img"].to(device), batch["label"].to(device)  # Changed 'image' to 'img'
        optimizer.zero_grad()
        if is_attacker:
            # Flip labels for attackers
            labels = (9 - labels)  # Simple label flipping
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Testing function with metric calculations
def test(net, testloader, device):
    """Validate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    net.to(device)
    net.eval()
    correct, loss = 0, 0.0
    all_preds = []
    all_labels = []
    all_outputs = []
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"].to(device), batch["label"].to(device)  # Changed 'image' to 'img'
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())
    accuracy = correct / len(testloader.dataset)

    # Compute metrics
    kappa = cohen_kappa_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    # Check if there are at least two classes in y_true
    unique_classes = np.unique(all_labels)
    if len(unique_classes) > 1:
        try:
            all_labels_bin = label_binarize(all_labels, classes=list(range(10)))
            all_outputs_array = np.array(all_outputs)
            roc_auc = roc_auc_score(all_labels_bin, all_outputs_array, average='macro', multi_class='ovr')
        except ValueError:
            roc_auc = float('nan')  # Assign NaN if ROC AUC cannot be computed
    else:
        roc_auc = float('nan')  # Assign NaN if only one class is present

    metrics = {
        "accuracy": accuracy,
        "kappa": kappa,
        "f1_score": f1,
        "roc_auc": roc_auc,
    }
    return loss, accuracy, metrics

# Define the FlowerClient class
class FlowerClient(NumPyClient):
    def __init__(self, trainloader, valloader, is_attacker=False) -> None:
        super().__init__()

        self.trainloader = trainloader
        self.valloader = valloader
        self.model = Net(num_classes=10)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.is_attacker = is_attacker

    def fit(self, parameters, config):
        """Train the model locally."""
        # Set model parameters
        set_params(self.model, parameters)

        # Define the optimizer
        optim = torch.optim.SGD(self.model.parameters(), lr=0.01, momentum=0.9)

        # Train the model
        train(self.model, self.trainloader, optim, self.device, is_attacker=self.is_attacker)

        # Return updated parameters
        return get_params(self.model), len(self.trainloader.dataset), {}

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]):
        """Evaluate the model locally."""
        set_params(self.model, parameters)
        loss, accuracy, metrics = test(self.model, self.valloader, self.device)
        return float(loss), len(self.valloader.dataset), metrics

# Define the client function
def client_fn(context: Context):
    """Create a Flower client representing a participant in the federated learning."""
    partition_id = int(context.node_config["partition-id"])
    partition = fds.load_partition(partition_id, "train")
    # Partition into train/validation
    partition_train_val = partition.train_test_split(test_size=0.1, seed=42)
    # Get dataloaders
    trainloader = get_cifar10_dataloaders(partition_train_val["train"], batch_size=BATCH_SIZE)  # Changed function name
    valloader = get_cifar10_dataloaders(partition_train_val["test"], batch_size=BATCH_SIZE)    # Changed function name
    # Determine if the client is an attacker
    is_attacker = partition_id in ATTACKER_IDS
    return FlowerClient(trainloader=trainloader, valloader=valloader, is_attacker=is_attacker).to_client()

# Define custom strategy to log metrics
class CustomFedAvg(FedAvg):
    def aggregate_evaluate(
        self,
        rnd: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ):
        """Aggregate evaluation results using weighted average and log metrics per round."""
        if not results:
            return None, {}
        
        # Use weighted average to aggregate metrics
        num_examples_total = sum([r[1].num_examples for r in results])

        # Initialize sums
        accuracy_sum = 0.0
        kappa_sum = 0.0
        f1_sum = 0.0
        roc_auc_sum = 0.0
        roc_auc_weight = 0

        for _, evaluate_res in results:
            accuracy_sum += evaluate_res.metrics.get("accuracy", 0.0) * evaluate_res.num_examples
            kappa_sum += evaluate_res.metrics.get("kappa", 0.0) * evaluate_res.num_examples
            f1_sum += evaluate_res.metrics.get("f1_score", 0.0) * evaluate_res.num_examples
            roc_auc = evaluate_res.metrics.get("roc_auc", float('nan'))
            if not np.isnan(roc_auc):
                roc_auc_sum += roc_auc * evaluate_res.num_examples
                roc_auc_weight += evaluate_res.num_examples

        # Compute weighted averages
        accuracy = accuracy_sum / num_examples_total
        kappa = kappa_sum / num_examples_total
        f1 = f1_sum / num_examples_total
        roc_auc = roc_auc_sum / roc_auc_weight if roc_auc_weight > 0 else float('nan')

        # Log metrics
        log(INFO, f"Round {rnd} evaluation metrics:")
        log(INFO, f"Accuracy: {accuracy:.4f}")
        log(INFO, f"Kappa: {kappa:.4f}")
        log(INFO, f"F1 Score: {f1:.4f}")
        if not np.isnan(roc_auc):
            log(INFO, f"ROC AUC: {roc_auc:.4f}")
        else:
            log(INFO, f"ROC AUC: Undefined (only one class present in y_true)")

        # Return aggregated loss and metrics
        return super().aggregate_evaluate(rnd, results, failures)

# Define the server function
def server_fn(context: Context):
    # Instantiate the model
    model = Net(num_classes=10)
    ndarrays = get_params(model)
    # Convert model parameters to flwr.common.Parameters
    global_model_init = ndarrays_to_parameters(ndarrays)

    # Define the strategy
    strategy = CustomFedAvg(
        fraction_fit=1.0,        # All clients participate in training
        fraction_evaluate=1.0,   # All clients participate in evaluation
        initial_parameters=global_model_init,  # Initialized global model
    )

    # Construct ServerConfig
    config = ServerConfig(num_rounds=NUM_ROUNDS)

    # Wrap everything into a ServerAppComponents object
    return ServerAppComponents(strategy=strategy, config=config)

# Create your ServerApp and ClientApp
server_app = ServerApp(server_fn=server_fn)
from flwr.client import ClientApp
client_app = ClientApp(client_fn=client_fn)

# Run the simulation
run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=NUM_CLIENTS,
    backend_name="ray",
    verbose_logging=True,
)


/home/fredrik/BTH/DV2607/part2/flower_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-22 18:15:55,875	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
DEBUG 2024-11-22 18:15:55,914:     Asyncio event loop already running.
DEBUG 2024-11-22 18:15:55,916:     Logger propagate set to False
DEBUG 2024-11-22 18:15:55,916:     Pre-registering run with id 16099003574598300099
DEBUG 2024-11-22 18:15:55,917:     Using InMemoryState
DEBUG 2024-11-22 18:15:55,918:     Using InMemoryState
DEBUG 2024-11-22 18:15:55,919:     Buffer time delay: 5s
INFO 2024-11-22 18:15:55,922:      Starting Flower ServerApp, config: num_rounds=50, no round_timeout
INFO 2024-11-22 18:15:55,923:      
INFO 2024-11-22 18:15:55,923: